<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/bert_multilingual_last_layer_dataset_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb

# Set up wandb

In [ ]:
import wandb

run = wandb.init(
    project="capture-disagreement",
    save_code = True,
)

# Load data

In [3]:
import pandas as pd
import os
import ast

artifact_dataset = run.use_artifact('capture_disagreement/capture-disagreement/dataset_combined_train:v0', type='dataset')
artifact_dir = artifact_dataset.download()
df_all = pd.read_csv(os.path.join(artifact_dir, "dataset_combined_train.csv"))
df_all['soft_label'] = df_all['soft_label'].apply(lambda x: ast.literal_eval(x))

wandb:   1 of 1 files downloaded.  


# Dataset

In [4]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label", "number of annotations", "annotations"]])

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/10431 [00:00<?, ?ex/s]

In [7]:
tokenized_datasets.set_format("torch")#not sure if neccessary


In [24]:
from torch.utils.data import DataLoader
batch_size = 8

In [9]:
from transformers import AutoModel
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [25]:
train_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=batch_size)

In [12]:
df_all

,text,annotation task,number of annotations,annotations,annotators,lang,hard_label,soft_label,split,other_info
0,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,misogyny and sexism detection in Arabic tweets,3,"0,0,0","Ann1,Ann2,Ann3",ar,0,"{'0': 1.0, '1': 0.0}",train,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
1,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,misogyny and sexism detection in Arabic tweets,3,"1,0,1","Ann1,Ann2,Ann3",ar,1,"{'0': 0.33, '1': 0.67}",train,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
2,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,misogyny and sexism detection in Arabic tweets,3,"1,1,1","Ann1,Ann2,Ann3",ar,1,"{'0': 0.0, '1': 1.0}",train,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
3,وقال تعالى الن الل احا الل تأملي هذة الأيه,misogyny and sexism detection in Arabic tweets,3,"0,0,0","Ann1,Ann2,Ann3",ar,0,"{'0': 1.0, '1': 0.0}",train,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
4,يعني السعوديات صايعات يتنقلو واحد للثاني,misogyny and sexism detection in Arabic tweets,3,"0,0,1","Ann1,Ann2,Ann3",ar,0,"{'0': 0.67, '1': 0.33}",train,"{'Ann1': 'Moderate_Female', 'Ann2': 'Liberal_F..."
...,...,...,...,...,...,...,...,...,...,...
10426,Instead of showing compassion and empathy..thi...,offensiveness detection,5,"0,0,0,0,0","Ann740,Ann577,Ann65,Ann775,Ann681",en,0,"{'0': 1.0, '1': 0.0}",train,{'domain': 'BLM'}
10427,<user> Neither did you,offensiveness detection,5,"0,0,0,0,0","Ann88,Ann85,Ann779,Ann793,Ann656",en,0,"{'0': 1.0, '1': 0.0}",train,{'domain': 'Elections2020'}
10428,"""University of Toronto, McMaster University, S...",offensiveness detection,5,"0,0,0,0,0","Ann632,Ann517,Ann514,Ann64,Ann609",en,0,"{'0': 1.0, '1': 0.0}",train,{'domain': 'Covid-19'}
10429,The only people ripping this country apart are...,offensiveness detection,5,"0,1,1,0,0","Ann493,Ann586,Ann239,Ann347,Ann661",en,0,"{'0': 0.6000000000000001, '1': 0.4}",train,{'domain': 'BLM'}


In [28]:
import torch


output_dir = 'dataset_embeddings'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the training data and save the model outputs
with torch.no_grad():
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):

    inputs, attention_mask, hard_labels, soft_labels, number_of_annotations, annotations = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"], batch["soft_label"], batch["number of annotations"], batch["annotations"]
    outputs = model(input_ids=inputs, attention_mask = attention_mask).pooler_output#.last_hidden_state#NllLoss

    for j in range(outputs.size(0)):
        output_path = os.path.join(output_dir, f'output_{i}_{j}.pt')
        data = {
            'output': outputs[j],
            'labels': hard_labels[j],
            'soft_labels': {'0': [soft_labels['1'][j]], '1': [soft_labels['1'][j]]},
            'num_annotations': number_of_annotations[j],
            'annotations': annotations[j]
            }
        torch.save(data, output_path)


<ipython-input-28-7e6e7f51a5d9>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):


  0%|          | 0/1304 [00:00<?, ?it/s]

In [32]:
class ModelOutputDataset(torch.utils.data.Dataset):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
    def __len__(self):
        return len(self.output_paths)
    
    def __getitem__(self, index):
        output_path = self.output_paths[index]

        data = torch.load(output_path)
        return data

In [33]:
bert_multilingual_last_layer_dataset = ModelOutputDataset(output_dir)

In [34]:
bert_multilingual_last_layer_dataset[0]['output'].shape

torch.Size([768])

In [4]:
artifact = wandb.Artifact('bert_multilingual_last_layer_dataset', type='dataset')
artifact.add_dir('dataset_embeddings') # Adds multiple files to artifact
run.log_artifact(artifact)

wandb: Adding directory to artifact (./dataset_embeddings)... Done. 2.0s


In [ ]:
run.finish()